# CarbonPlan Reports API Demo

This notebook demonstrates how to use CarbonPlan's API to access and analyze project reports.

In [ ]:
import requests
import pandas as pd
import altair as alt

from IPython.display import JSON

### Schema

All API access is over HTTPS, and accessed from https://api.carbonplan.now.sh. All data is sent and received as JSON. The full schema is available via the API: 

In [ ]:
schema = requests.get('https://api.carbonplan.now.sh/schema/').json()

# list objects defined in CarbonPlan's schema
schema['objects']
JSON(schema)

In [ ]:
# or get the full schema for a specific object
schema = requests.get('https://api.carbonplan.now.sh/schema/ProjectCollection.json').json()
JSON(schema)

### Projects

CarbonPlan's full ProjectCollection is available at https://api.carbonplan.now.sh/projects:

In [ ]:
project_collection = requests.get('https://api.carbonplan.now.sh/projects').json()
JSON(project_collection, root='project_collection')

Individual projects can be requested using query parameters. In the example below, we point our API request at `id=STRP01`:

In [ ]:
one_project = requests.get('https://api.carbonplan.now.sh/projects?id=STRP01').json()
JSON(one_project)

### Sample Analysis

Now that we've gone over how to access CarbonPlan's API, we can turn to some sample analysis. Here we'll simply take the full project collection (defined above) and dump it into a `pandas.DataFrame`.

In [ ]:
projects = pd.DataFrame.from_dict(project_collection['projects'], ).set_index('id')
projects.head()

You'll notice some columns still have nested JSON in them so we'll want to unpack them further. In the next cell we unpack the metrics into a `DataFrame` of its own.

In [ ]:
metrics = [m['name'] for m in projects.iloc[0].metrics]

def get_metric(row, name='volume', key='value'):
    ''' helper function to extract metric values by name '''
    for m in row.metrics:
        if m['name'] == name:
            return m[key]

metrics_df = pd.DataFrame({m: projects.apply(get_metric, name=m, axis=1) for m in metrics}).apply(pd.to_numeric, errors='coerce')
metrics_df['tag'] = projects['tags'].apply(lambda x: x[0])
metrics_df['name'] = projects['name']
metrics_df.head()

From here, we can start doing some real analysis and data visualization with our metrics `DataFrame`.

In the section below, we simply make the three charts that are shown on https://reports.carbonplan.org. We use [Altair](https://altair-viz.github.io/) to make a few interactive vizualizations.

In [ ]:
colors = {
    'mineralization': '#a9b4c4',
    'soil': '#ea9755',
    'biomass': '#d4c05e',
    'forests': '#7eb36a',
    'ocean': '#64b9c4',
    'dac': '#bc85d9'
}
color_scale = alt.Color('tag:N', scale=alt.Scale(domain=list(colors.keys()), range=list(colors.values())))
tooltip = ['name', 'volume', 'cost', 'permanence', 'negativity']

In [ ]:
# selectors
brush = alt.selection_interval(encodings=['x', 'y'])
click = alt.selection_multi()

# chart
chart = alt.Chart(metrics_df).mark_circle(size=200).encode(
    color=color_scale,
    opacity=alt.condition(brush | click, alt.OpacityValue(1), alt.OpacityValue(0.4)),
    tooltip=tooltip
).properties(
    width=250,
    height=250
).add_selection(
    brush, click
)

# hconcat
(
    chart.encode(
        alt.X('cost', axis=alt.Axis(title='COST $/tCO2', tickCount=3), scale=alt.Scale(type='log', domain=[2, 2000], nice=False)),
        alt.Y('tag', axis=alt.Axis(title=''))
    ) |
    chart.encode(
        alt.X('volume', axis=alt.Axis(title='VOLUME tCO2', tickCount=3), scale=alt.Scale(type='log', domain=[2, 100000000])),
        alt.Y('tag', axis=alt.Axis(title=''))
    ) |
    chart.encode(
        alt.X('negativity', axis=alt.Axis(title='NEGATIVITY', tickCount=2), scale=alt.Scale(domain=[-0.1, 1.1])),
        alt.Y('tag', axis=alt.Axis(title=''))
    ) |
    chart.encode(
        alt.X('permanence', axis=alt.Axis(title='PERMANENCE years', tickCount=3), scale=alt.Scale(type='log', domain=[0.6, 2000], nice=False)),
        alt.Y('tag', axis=alt.Axis(title='')))
).configure_axis(grid=False).resolve_scale(
    y='shared'
)

In [ ]:
# selectors
brush = alt.selection_interval(encodings=['x', 'y'])
click = alt.selection_multi()

# chart
chart = alt.Chart(metrics_df).mark_circle(size=200).encode(
    color=color_scale,
    opacity=alt.condition(brush | click, alt.OpacityValue(1), alt.OpacityValue(0.4)),
    tooltip=tooltip
).properties(
    width=250,
    height=250
).add_selection(
    brush, click
)

# hconcat
(
    chart.encode(
        alt.X('volume', axis=alt.Axis(title='VOLUME tCO2', tickCount=3), scale=alt.Scale(type='log', domain=[2, 100000000])),
        alt.Y('cost', axis=alt.Axis(title='COST $/tCO2', tickCount=3), scale=alt.Scale(type='log', domain=[2, 2000], nice=False)),
    ) |
    chart.encode(
        alt.X('negativity', axis=alt.Axis(title='NEGATIVITY', tickCount=2), scale=alt.Scale(domain=[-0.1, 1.1])),
        alt.Y('tag', axis=alt.Axis(title=''))
    ) |
    chart.encode(
        alt.X('permanence', axis=alt.Axis(title='PERMANENCE years', tickCount=3), scale=alt.Scale(type='log', domain=[0.6, 2000], nice=False)),
        alt.Y('tag', axis=alt.Axis(title='')))
).configure_axis(grid=False).resolve_scale(
    y='shared'
)